###1.tensors使用
tensor 类似numpy的ndarray，唯一的区别是Tensor可以在GPU上加速运算。
文档地址：https://pytorch.org/docs/torch

In [20]:
from __future__ import print_function
import torch

#创建空矩阵
x = torch.empty(5,3)
print(x)



tensor([[1.5414e-44, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])


In [21]:
#随机矩阵
x = torch.rand(4,4)
print(x)

tensor([[0.0752, 0.7042, 0.5692, 0.6063],
        [0.0103, 0.2387, 0.0026, 0.4889],
        [0.3450, 0.0255, 0.8103, 0.1775],
        [0.0737, 0.4501, 0.2954, 0.5125]])


In [22]:
#long型全0矩阵
x = torch.zeros(4,4,dtype=torch.long)
print(x)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])


In [23]:
#数组直接构建tensor
x = torch.tensor([1.0,2,3,4])
print(x)

tensor([1., 2., 3., 4.])


从一个已有的tensor构建一个tensor。这些方法会重用原来tensor的特征，例如，数据类型，除非提供新的数据。

In [24]:
x = x.new_ones(4,3,dtype=torch.double) #
print(x)

x = torch.randn_like(x,dtype=torch.float) #修改类型
print(x) #size 跟以前一样

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.9760,  0.5724,  0.6203],
        [-0.2956,  0.3880, -1.7083],
        [-0.1816, -0.3425,  1.1022],
        [-1.5756,  0.0491, -0.9492]])


得到tensor形状

In [25]:
print(x.size())

torch.Size([4, 3])


运算

In [26]:
#加法运算, 两种方式
y = torch.rand(4,3)

print(x+y)

print(torch.add(x,y))

#加法：把输出作为变量

result = torch.empty(4,3)
torch.add(x,y,out= result)
print(result)


tensor([[ 1.1665,  1.2073,  0.6982],
        [ 0.0755,  0.8652, -1.1181],
        [ 0.1489,  0.3487,  1.3202],
        [-0.8148,  0.7755, -0.6853]])
tensor([[ 1.1665,  1.2073,  0.6982],
        [ 0.0755,  0.8652, -1.1181],
        [ 0.1489,  0.3487,  1.3202],
        [-0.8148,  0.7755, -0.6853]])
tensor([[ 1.1665,  1.2073,  0.6982],
        [ 0.0755,  0.8652, -1.1181],
        [ 0.1489,  0.3487,  1.3202],
        [-0.8148,  0.7755, -0.6853]])


in-place加法:y=x+y,结果返回到y上 

In [27]:
y.add_(x)
print(y)

tensor([[ 1.1665,  1.2073,  0.6982],
        [ 0.0755,  0.8652, -1.1181],
        [ 0.1489,  0.3487,  1.3202],
        [-0.8148,  0.7755, -0.6853]])


注意：
任何in-place的运算都会以``_``结尾。 举例来说：``x.copy_(y)``, ``x.t_()``, 会改变 ``x``。

各种类似NumPy的indexing都可以在PyTorch tensor上面使用。

In [28]:
print(x[:2,1:])

tensor([[ 0.5724,  0.6203],
        [ 0.3880, -1.7083]])


Resizing: 如果你希望resize/reshape一个tensor，可以使用torch.view：

In [29]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


如果你有一个只有一个元素的tensor，使用.item()方法可以把里面的value变成Python数值。

In [30]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.0489])
0.04888658970594406


###numpy和tensor之间转换

Torch Tensor和NumPy array会共享内存，所以改变其中一项也会改变另一项。

把Torch Tensor转变成NumPy Array

In [31]:
a = torch.ones(8)
print(a)

tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [32]:
b = a.numpy() #转numpy
print(b) 

[1. 1. 1. 1. 1. 1. 1. 1.]


改变numpy array里面的值。

In [33]:
a.add_(2)
print(a)
print(b)

tensor([3., 3., 3., 3., 3., 3., 3., 3.])
[3. 3. 3. 3. 3. 3. 3. 3.]


把NumPy ndarray转成Torch Tensor

In [34]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a,2,out = a)
print (a)
print(b)

[3. 3. 3. 3. 3.]
tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


所有CPU上的Tensor都支持转成numpy或者从numpy转成Tensor。

###CUDA Tensors
使用.to方法，Tensor可以被移动到别的device上。

In [36]:
print(torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x,device=device) #在GPU上创建tensor
    
    x = x.to(device)  #转到gpu，or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu",torch.double)) #转成cpu，并改变type
    


False


### 用numpy实现两层神经网络
一个全连接ReLU神经网络，一个隐藏层，没有bias。用来从x预测y，使用L2 Loss。

这一实现完全使用numpy来计算前向神经网络，loss，和反向传播。

numpy ndarray是一个普通的n维array。它不知道任何关于深度学习或者梯度(gradient)的知识，也不知道计算图(computation graph)，只是一种用来计算数学运算的数据结构。

In [39]:
import numpy as np
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N,D_in,H,D_out = 64,1000,100,10

#创建随机数据
x = np.random.randn(N,D_in)
y = np.random.randn(N,D_out)

#随机初始化权重 Randomly initialize weights
w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H,D_out)


learning_rate = 1e-6

for t in range(500):
    
    #1.前向传播 Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h,0)
    y_hat =h_relu.dot(w2)
    
    #2.计算损失；均方差 compute and print loss
    loss = np.square(y_hat - y).sum()
    if t %10 ==0:
        print(t,loss)
    
    #3.反向传播
    # Backprop to compute gradients of w1 and w2 with respect to loss
    
    
    grad_y_hat =2.0 * (y_hat - y)
    
    grad_w2 = h_relu.T.dot(grad_y_hat)
    grad_h_relu = grad_y_hat.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    #update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    


0 23682262.069874708
10 2844937.626818004
20 262312.2981393648
30 84135.25461929964
40 32815.93217014946
50 14136.731450632218
60 6558.356400944219
70 3234.5730572430703
80 1681.5777304929707
90 915.8213042072607
100 518.6571628697882
110 302.71803458185946
120 180.87878165963372
130 110.04122786591489
140 67.88573820504426
150 42.339343352743015
160 26.631030006956045
170 16.872118294537508
180 10.748390066405733
190 6.878742688237029
200 4.419273617723595
210 2.848639742213871
220 1.841573645183747
230 1.1935801058040174
240 0.7753655156719301
250 0.5047252585607946
260 0.3291653631447625
270 0.21503816356788477
280 0.14070238379277689
290 0.09219582083955112
300 0.06049345366282413
310 0.039740955340409466
320 0.026138084506863165
330 0.017209642978849957
340 0.011342294372491568
350 0.007482423193660584
360 0.004940461327461458
370 0.0032647366750481075
380 0.0021590559177970276
390 0.001428904316704597
400 0.000946326849952885
410 0.0006271347615142479
420 0.0004158597886357795
43

###pytorch:tensors 实现两层神经网络

我们使用PyTorch tensors来创建前向神经网络，计算损失，以及反向传播。

一个PyTorch Tensor很像一个numpy的ndarray。但是它和numpy ndarray最大的区别是，PyTorch Tensor可以在CPU或者GPU上运算。如果想要在GPU上运算，就需要把Tensor换成cuda类型。



In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")
#device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

#create random input and output

x = torch.randn(N,D_in,device=device,dtype = dtype)
y = torch.randn(N,D_out,device = device,dtype = dtype)

#randomly initialize weights
w1 = torch.randn(D_in,)










